In [ ]:
from __future__ import annotations

from minigrid.core.grid import Grid
from minigrid.core.mission import MissionSpace
from minigrid.core.world_object import Goal, Wall
from minigrid.minigrid_env import MiniGridEnv
from minigrid.wrappers import SymbolicObsWrapper


class SimpleEnv(MiniGridEnv):
    def __init__(
        self,
        size=5,
        agent_start_pos=(1, 1),
        agent_start_dir=0,
        max_steps: int | None = None,
        **kwargs,
    ):
        self.agent_start_pos = agent_start_pos
        self.agent_start_dir = agent_start_dir

        mission_space = MissionSpace(mission_func=self._gen_mission)

        if max_steps is None:
            max_steps = 4 * size**2

        super().__init__(
            mission_space=mission_space,
            grid_size=size,
            see_through_walls=True,
            max_steps=max_steps,
            **kwargs,
        )

    @staticmethod
    def _gen_mission():
        return "grand mission"

    def _gen_grid(self, width, height):
        self.grid = Grid(width, height)

        # surrounding walls
        self.grid.wall_rect(0, 0, width, height)

        # vertical wall
        for i in range(0, height - 2):
            self.grid.set(2, i, Wall())

        # goal
        self.put_obj(Goal(), width - 2, height - 2)

        # agent
        self.agent_pos = self.agent_start_pos
        self.agent_dir = self.agent_start_dir

        self.mission = "grand mission"


def main():
    env = SimpleEnv(render_mode="human")
    obs, _ = env.reset()
    #print("Observation keys:", obs.keys())
    #print("Symbolic state (first 10 entries):")
    print(obs["image"].shape)
    print("Agent direction:", obs["direction"])
    print("Mission:", obs["mission"])

    env = SymbolicObsWrapper(env)

    obs, _ = env.reset()
    print(type(env))

    print("Observation keys:", obs.keys())
    print("Symbolic state (first 10 entries):")
    print(obs["image"].shape)
    print("Agent direction:", obs["direction"])
    print("Mission:", obs["mission"])
    print(obs["image"])

if __name__ == "__main__":
    main()

(7, 7, 3)
Agent direction: 0
Mission: grand mission
<class 'minigrid.wrappers.SymbolicObsWrapper'>
Observation keys: dict_keys(['image', 'direction', 'mission'])
Symbolic state (first 10 entries):
(5, 5, 3)
Agent direction: 0
Mission: grand mission
[[[ 0  0  2]
  [ 0  1  2]
  [ 0  2  2]
  [ 0  3  2]
  [ 0  4  2]]

 [[ 1  0  2]
  [ 1  1 10]
  [ 1  2 -1]
  [ 1  3 -1]
  [ 1  4  2]]

 [[ 2  0  2]
  [ 2  1  2]
  [ 2  2  2]
  [ 2  3 -1]
  [ 2  4  2]]

 [[ 3  0  2]
  [ 3  1 -1]
  [ 3  2 -1]
  [ 3  3  8]
  [ 3  4  2]]

 [[ 4  0  2]
  [ 4  1  2]
  [ 4  2  2]
  [ 4  3  2]
  [ 4  4  2]]]


: 